In [1]:
import MeCab
import re
from gensim import corpora
from gensim import matutils
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import TfidfTransformer

In [2]:
file_path = ["/Users/nakazawamika/Documents/研究室/オフィス/データ/5-18_normal.txt",
             "/Users/nakazawamika/Documents/研究室/オフィス/データ/5-25_normal.txt",
             "/Users/nakazawamika/Documents/研究室/オフィス/データ/5-26_normal.txt",
             "/Users/nakazawamika/Documents/研究室/オフィス/データ/5-31_normal.txt",
             "/Users/nakazawamika/Documents/研究室/オフィス/データ/6-2_normal.txt",
             "/Users/nakazawamika/Documents/研究室/オフィス/データ/6-5_normal.txt",
             "/Users/nakazawamika/Documents/研究室/オフィス/データ/6-6_normal.txt",
             "/Users/nakazawamika/Documents/研究室/オフィス/データ/6-9_normal.txt",
             "/Users/nakazawamika/Documents/研究室/オフィス/データ/6-12_normal.txt",
             "/Users/nakazawamika/Documents/研究室/オフィス/データ/5-15_ゼミのコピー.txt",
             "/Users/nakazawamika/Documents/研究室/オフィス/データ/5-16_ゼミのコピー.txt",
             "/Users/nakazawamika/Documents/研究室/オフィス/データ/5-22_ゼミのコピー.txt",
             "/Users/nakazawamika/Documents/研究室/オフィス/データ/6-5_ゼミのコピー.txt",
             "/Users/nakazawamika/Documents/研究室/オフィス/データ/6-6_ゼミのコピー.txt",
             "/Users/nakazawamika/Documents/研究室/オフィス/データ/6-12_ゼミのコピー.txt",
             "/Users/nakazawamika/Documents/研究室/オフィス/データ/6-13_ゼミのコピー.txt",
             "/Users/nakazawamika/Documents/研究室/オフィス/データ/7-3_ゼミのコピー.txt",
             "/Users/nakazawamika/Documents/研究室/オフィス/データ/7-4_ゼミのコピー.txt",
             "/Users/nakazawamika/Documents/研究室/オフィス/データ/7-11_ゼミのコピー.txt"]

#ファイルを開く　with：ファイルを自動的に閉じてくれる　r：読み込み 
paragraphs = []
for file_list in file_path:
    with open(file_list, "r", encoding="utf-8", errors="replace") as file:  
        paragraphs.append(file.read())

#↓リスト内包表記
# paragraphs = [open(file_paths, 'r', encoding='utf-8').read() for file_paths in file_path]
# print(type(paragraphs)) #list型

In [3]:
# ストップワードファイルの読み込み
with open('./more_stopword.txt') as f:
    stop_words = f.read().splitlines()


In [4]:
#テキスト整形
text = []
for text_format in paragraphs:
    text_format = re.sub(r'[!"#$%&\'\\\\()*+,-./:;<=>?@[\\]^_`{|}~「」〔〕“”〈〉『』【】＆＊・（）＄＃＠。、？！｀＋￥％]', '', text_format)
    text_format = re.sub(r'\d', "", text_format) #数字削除
    text_format = text_format.lower() #小文字へ
    text.append(text_format)

In [5]:
words_list = []
words_in_sentence = []
tagger = MeCab.Tagger()
# sentence_parsed = tagger.parse(text[0])
# print(sentence_parsed)
for sentence in text:
    result_word_list = []
    sentence_parsed = tagger.parse(sentence)
    for line in sentence_parsed.splitlines()[:-1]:
        if '名詞' in line and re.search('^[ぁ-んァ-ン一-龥]+', line):  #search or findall? 
            word = line.split('\t')[0]
            # result_word_list.append(line.split('\t')[0])
            if not word in stop_words:  # ストップワードの削除
                result_word_list.append(word)
    words_list.append(result_word_list)

In [6]:
# 単語とユニーク番号の辞書
word2int = {}
i = 0
for words in words_list:
    for word in words:
        if word not in word2int:
            word2int[word] = i
            i += 1

In [7]:
# 単語の出現回数をデータフレーム化
word2int_gs = corpora.Dictionary(words_list)
bow_gs = np.array([matutils.corpus2dense([word2int_gs.doc2bow(words)], num_terms=len(word2int)).T[0] for words in words_list]).astype(np.int64)
bow_gs_df = pd.DataFrame(bow_gs, columns=list(word2int_gs.values()))

display(bow_gs_df)

,ご覧,ご飯,でんぷん,なり,なわ,ねびき,ねぶり,のど,のの,はいはい,...,開大,防護,隕筋,面積,響き,頭髪,頻度,飼い,首相,齧
0,1,5,1,1,1,2,1,1,1,1,...,0,0,0,0,0,0,0,0,0,0
1,2,5,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,4,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,6,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,1,2,0,0,1,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
5,1,6,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6,1,6,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7,0,3,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8,1,3,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9,2,0,0,3,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [8]:
tfidf = TfidfTransformer(use_idf=True, norm='l2', smooth_idf=True)
tfidf_sk = tfidf.fit_transform(bow_gs).toarray()
tfidf_sk_df = pd.DataFrame(tfidf_sk, columns=list(word2int_gs.values()))

display(tfidf_sk_df)

,ご覧,ご飯,でんぷん,なり,なわ,ねびき,ねぶり,のど,のの,はいはい,...,開大,防護,隕筋,面積,響き,頭髪,頻度,飼い,首相,齧
0,0.007877,0.031557,0.015364,0.007877,0.011101,0.030728,0.015364,0.015364,0.015364,0.013478,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,0.016723,0.033499,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2,0.030317,0.006073,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
3,0.000000,0.038957,0.000000,0.008103,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
4,0.007575,0.012139,0.000000,0.000000,0.010676,0.000000,0.000000,0.000000,0.000000,0.012961,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
5,0.015185,0.073006,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
6,0.008989,0.043216,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
7,0.000000,0.018645,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
8,0.008956,0.021529,0.000000,0.008956,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
9,0.014902,0.000000,0.000000,0.022353,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


In [9]:
#csv
tfidf_sk_df.to_csv('/Users/nakazawamika/Documents/test/TF-IDF/sample_normal.csv', index=False, encoding="shift-jis", errors="ignore")

In [ ]:
# wb.save('data/dst/openpyxl_sample.xlsx')